## Plan of action.
We have too little data to train a model dircetly on the data. It will either be too stupid or terribly overfit.

To work with this, we will use different data, which is label with that slighly correspond to the labels we are interested in. We will train the net, then remove the last layers then and add new layers ontop. Then finetune.

Ideas:
1) don't use pooling but stride to reduce amount of parameters.
2) normailze
3) batch norm
4) dropout

In [58]:
import importlib
import GetData
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

importlib.reload(GetData)

train_loader, test_loader = GetData.load_pretrain_data(batch_size = 64)

In [59]:
device = torch.device("cuda.0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [60]:
# train loop
def train_model(model, data_loader, epochs, lr=0.1, optim=None):
  if optim is None:
    optimizer = torch.optim.SGD(params=model.parameters(), lr=lr)
  else:
    optimizer = optim
  loss_fn = nn.MSELoss()
  
  model.to(device)
  for epoch in range(epochs):
    epoch_loss = 0
    for batch, (X, y) in enumerate(data_loader):
      X = X.to(device)
      y = y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      epoch_loss += loss.item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    print(f"average batch loss in {epoch+1}: {epoch_loss/len(data_loader)}")
# test loop
def test_model(model, data_loader):
  loss_fn = nn.MSELoss() 
  model.to(device)
  with torch.no_grad():
    for batch, (X,y) in enumerate(data_loader):
      X = X.to(device)
      y = y.to(device)
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
    print(f"average batch loss: {loss.item()/len(data_loader)}")

In [70]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=6, stride=2, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=6, out_channels=16, stride=2, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=120, stride=2, kernel_size=3)
        self.fc1 = nn.Linear(120*124, 84)
        self.fc2 = nn.Linear(84, 1)

    def forward(self, x):
        x = x.unsqueeze_(dim=1)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 120*124)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze_(dim=1)
        return x
    
# using batch normalization
class normalized_net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=6, stride=2, kernel_size=3)
        self.bn1 = nn.BatchNorm1d(6)
        self.conv2 = nn.Conv1d(in_channels=6, out_channels=16, stride=2, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(16)
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=120, stride=2, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(120)
        self.fc1 = nn.Linear(120*124, 84)
        self.fc2 = nn.Linear(84, 1)

    def forward(self, x):
        x = x.unsqueeze_(dim=1) # need the x = x.unsqueeze_(dim=1) so gradient computation works
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = torch.relu(self.bn3(self.conv3(x)))
        x = x.view(-1, 120*124)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze_(dim=1)    
        return x


In [65]:
test = next(iter(train_loader))[0]
dev_model = net()
out = dev_model(test)
print(out.shape)

torch.Size([64])


In [ ]:
dev_model = net()
test_model(dev_model, train_loader)
train_model(dev_model, train_loader, epochs=10, lr=0.01)